In [1]:
{-# LANGUAGE InstanceSigs #-}
import Control.Applicative (Alternative(..))
import Control.Monad (guard)
import Data.Char (isSpace, isDigit, ord)

newtype Parser a = Parser { runParser :: String -> Maybe (a, String) }

In [2]:
instance Functor Parser where
    fmap :: (a -> b) -> Parser a -> Parser b
    fmap f parser = Parser $ \s -> let
        fn (a, s) = (f a, s)
        in fn <$> runParser parser s

In [3]:
instance Applicative Parser where
    pure :: a -> Parser a
    pure a = Parser $ \s -> Just (a, s)
    (<*>) :: Parser (a -> b) -> Parser a -> Parser b
    f <*> a = Parser $ \s -> case runParser f s of
        Nothing -> Nothing
        Just (f', s') -> case runParser a s' of
            Nothing -> Nothing
            Just (a', s'') -> Just (f' a', s'')

instance Alternative Parser where
    empty :: Parser a
    empty = Parser $ \_ -> Nothing
    
    (<|>) :: Parser a -> Parser a -> Parser a
    a <|> b = Parser $ \s -> runParser a s <|> runParser b s

choice :: [Parser a] -> Parser a
choice [] = empty
choice (p:ps) = p <|> choice ps

In [4]:
instance Monad Parser where
    (>>=) :: Parser a -> (a -> Parser b) -> Parser b
    a >>= f = Parser $ \s -> case runParser a s of
        Nothing       -> Nothing
        Just (a', s') -> runParser (f a') s'

In [5]:
anyChar :: Parser Char
anyChar = Parser $ \s -> case s of
    []     -> Nothing
    (c:cs) -> Just (c, cs)

satisfy :: (Char -> Bool) -> Parser Char
satisfy pred = do
    c <- anyChar
    guard $ pred c
    pure c

char :: Char -> Parser Char
char c = satisfy (c==)

string :: String -> Parser String
string []     = pure []
string (c:cs) = (:) <$> char c <*> string cs

In [6]:
sepBy :: Parser a -> Parser b -> Parser [a]
sepBy p sep = (p `sepBy1` sep) <|> pure []

sepBy1 :: Parser a -> Parser b -> Parser [a]
sepBy1 p sep = (:) <$> p <*> many (sep *> p)

In [7]:
chainl :: Parser a -> Parser (a -> a -> a) -> a -> Parser a
chainl p op a = (p `chainl1` op) <|> pure a

chainl1 :: Parser a -> Parser (a -> a -> a) -> Parser a
chainl1 p op = p >>= rest
    where rest a = (do
            f <- op
            b <- p
            rest (f a b)) <|> pure a

chainr :: Parser a -> Parser (a -> a -> a) -> a -> Parser a
chainr p op a = (p `chainr1` op) <|> pure a

chainr1 :: Parser a -> Parser (a -> a -> a) -> Parser a
chainr1 p op = scan
    where
        scan   = p >>= rest
        rest a = (do
            f <- op
            b <- scan
            rest (f a b)) <|> pure a

In [8]:
space :: Parser String
space = many (satisfy isSpace)

token :: Parser a -> Parser a
token p = p <* space

symb :: String -> Parser String
symb = token . string

apply :: Parser a -> String -> Maybe (a, String)
apply p = runParser (space *> p)

In [9]:
expr, term, factor, digit :: Parser Int
expr   = term   `chainl1` addop
term   = factor `chainl1` mulop
factor = digit <|> (symb "(" *> expr <* symb ")")
digit  = subtract (ord '0') . ord <$> token (satisfy isDigit)

addop, mulop :: Parser (Int -> Int -> Int)
addop = (symb "+" *> pure (+)) <|> (symb "-" *> pure (-))
mulop = (symb "*" *> pure (*)) <|> (symb "/" *> pure (div))

In [10]:
runParser expr "(1 + 2 * 4) / 3"

Just (3,"")